# 1 研究问题

## 1.1 研究背景  

 * **虚假信息指的是那些捏造的新闻报道、虚假的谣言、阴谋论或错误信息等(Lewandowsky et al., 2021)。**  
    这些信息的广泛传播对社会产生严重的负面影响，例如导致“疫苗犹豫”等现象。因此，近年来研究者们对理解虚假信息信念的心理机制以及如何减少其传播产生了浓厚的兴趣。  
		
![Image Name](https://cdn.kesci.com/upload/s6ko6z2vnb.jpg?imageView2/0/w/960/h/960)  

  
* **个体在判断信息真伪时，存在显著的党派差异。人们更容易相信与自己政治立场一致的新闻或来源于相同党派的消息源。**  
   这种党派分歧的存在有多种可能的原因。  

* **一种解释认为，人们的判断经常由追求准确性驱动（即准确性动机），但同时他们也受到社会动机的引导，如群体归属和地位等。这些社会动机可能会干扰准确性动机(Taber & Lodge, 2006)。另一种解释认为，由于接触不同党派新闻渠道和社交媒体，党派成员具有不同的先前知识或信念(Pennycook & Rand, 2021)。**  
   然而，除非通过实验操纵准确性动机或社会动机，否则很难区分这两种解释。如果虚假信息信念在某种程度上反映了动机因素，那么实验操纵人们的准确性动机或社会动机应该会改变他们对虚假信息的判断。相反，如果虚假信息信念仅仅反映了个体不同的先前信念，那么这些实验操纵应该不会改变他们对虚假信息的判断。  


## 1.2 研究设计  
* **为了进一步探讨这一问题，研究者进行了一系列实验。通过提供正确识别标题的金钱激励，研究者检验了准确性动机在政治新闻真假判断中的作用。**  
* **同时，还检验了当从帖子中移除党派来源线索时，金钱激励的作用是否会减弱。**  

* **自变量**：准确性激励（准确性激励/控制）、信息源（有信息源/无信息源）  
   **因变量**：真假判断的准确性  

* **假设1**：准确性激励影响被试判断的准确性，存在准确性激励时判断更准确  
   **假设2**：信息源影响被试判断的准确性，有信息源时的判断更准确  
	 **假设3**：准确性激励和信息源对被试判断准确性的影响存在交互作用

## 1.3 录入数据与描述性统计

In [1]:
# 导入所需的所有工具包
import pymc as pm
import arviz as az
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import ipywidgets
import bambi as bmb
from scipy import stats

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 读取数据
df = pd.read_csv('/home/mw/input/data2675/last2.csv')

 * **原作者建立了一个算术式作为衡量被试判断准确性的指标，即因变量**  
       但是原本的因变量有正有负，在采取了连接函数后还是不能做到很好的拟合  
			 
 * **因此我们将因变量改为被试判断结果为真或假，设置题目本身真或假为自变量**  
       可以看到：在正确判断的情况下，题目本身的真假会对因变量存在主效应（即题目为真，被试更倾向于判断为真）  
       而题目本身真假与其他自变量的交互作用可以展现其他自变量对被试判断准确性影响的指标（如果存在正向的交互作用，说明该自变量能提升被试判断的准确性

In [3]:
# 选取本节课涉及的变量
df = df[["Accuracy","Source","timu","value"]]

# Accurracy 题目是否有激励  -0.5无0.5有
df["Accuracy"] =  np.where(df['Accuracy'] == "Control", -0.5, 0.5)
# Source 是否有题目来源  -0.5无0.5有 
df["Source"] =  np.where(df['Source'] == "No Source", -0.5, 0.5)
# timu 题目本身的真假 -0.5假0.5真
df["timu"] =  np.where(df['timu'] == "F", -0.5, 0.5)
# value 被试判断的真假，0假1真

#设置索引
df["index"] = range(len(df))
df = df.set_index("index")

#由于数据量过大，删除部分数据
df = df.loc[5768:8968]
df

,Accuracy,Source,timu,value
index,,,,
5768,-0.5,-0.5,-0.5,0
5769,-0.5,-0.5,-0.5,0
5770,-0.5,-0.5,-0.5,0
5771,-0.5,-0.5,-0.5,1
5772,-0.5,-0.5,-0.5,1
...,...,...,...,...
8964,-0.5,0.5,0.5,1
8965,-0.5,0.5,0.5,0
8966,-0.5,0.5,0.5,1



![Image Name](https://cdn.kesci.com/upload/s6mcucssfy.png?imageView2/0/w/960/h/960)  

![Image Name](https://cdn.kesci.com/upload/s6mcusjn9a.png?imageView2/0/w/960/h/960)  


* **🤔此处为什么要用-0.5,0.5编码，而不是常规的0，1编码？**  
  编码方式不同，当自变量取特定值的情况下，其与beta组合成的表达式也会发生变化  
	各自变量的主效应、交互效应由表达式计算而得  
	**因此，自变量的编码方式会影响beta所代表的含义**

* **由于前文对数据进行了筛选，我们对自变量的分布情况进行了检验，以考察截取部分的数据是否包含了全部处理条件**  
  由下图可见，截取部分的数据包含了准确性激励（有信息源）、控制（有信息源）、准确性激励（无信息源）和控制（无信息源）四种处理条件

In [4]:
plt.hist(df["Accuracy"])
plt.xticks([-0.5,0.5])
plt.xlabel("accuracy")
plt.ylabel("count")
sns.despine()
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
plt.hist(df["Source"])
plt.xticks([-0.5,0.5])
plt.xlabel("source")
plt.ylabel("count")
sns.despine()
plt.show()

<Figure size 640x480 with 1 Axes>

In [6]:
Source = ["Source", "NoSource"]
Accuracy = [784,736]
Control = [808,873]


# 创建分组柱状图，需要自己控制x轴坐标
xticks = np.arange(len(Source))

fig, ax = plt.subplots(figsize=(5, 3.5))

ax.bar(xticks, Accuracy, width=0.25, label="Accuracy", color="#ff7f0e")
ax.bar(xticks + 0.25, Control, width=0.25, label="Control", color="#1f77b4")


ax.set_xlabel("Source")
ax.set_ylabel("count")
ax.legend()

# 最后调整x轴标签的位置
ax.set_xticks(xticks+0.125)
ax.set_xticklabels(Source)

[Text(0.125, 0, 'Source'), Text(1.125, 0, 'NoSource')]

<Figure size 500x350 with 1 Axes>

In [7]:
Source = ["Source", "NoSource"]
Accuracy = [68.4,65.4]
Control = [63.8, 62.1]


# 创建分组柱状图，需要自己控制x轴坐标
xticks = np.arange(len(Source))

fig, ax = plt.subplots(figsize=(5, 3.5))

ax.bar(xticks, Accuracy, width=0.25, label="Accuracy", color="#ff7f0e")
ax.bar(xticks + 0.25, Control, width=0.25, label="Control", color="#1f77b4")


ax.set_title("ACC", fontsize=15)
ax.set_xlabel("Source")
ax.set_ylabel("ACC")
ax.legend()

# 最后调整x轴标签的位置
ax.set_xticks(xticks+0.125)
ax.set_xticklabels(Source)

[Text(0.125, 0, 'Source'), Text(1.125, 0, 'NoSource')]

<Figure size 500x350 with 1 Axes>

总体数据的描述性统计

# 2 模型定义与搭建  

## 2.1 模型定义

* **model1: (激励的作用，指标为激励和题目本身对被试判断的交互效应)**  
$$   
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1,\beta_2,\beta_4 & \stackrel{ind}{\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 + \beta_1 X_{i1} + \beta_2 X_{i2} + \beta_4 X_{i1} X_{i2}}}{1 + e^{\beta_0 + \beta_1 X_{i1} + \beta_2 X_{i2} + \beta_4 X_{i1} X_{i2}}} \\  
\text{priors:} & & \beta_{0}  &  \sim N\left(0, 0.5^2 \right)  \\  
               & & \beta_1  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_2  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_4  & \sim N\left(0, 0.5^2 \right)\\  
\end{array}  
$$  

* **model2 : (来源的作用，指标为来源和题目本身对被试判断的交互效应)**  
$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_2,\beta_3,\beta_5 & \stackrel{ind}{\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 +   \beta_3 X_{i3} + \beta_2 X_{i2} + \beta_5 X_{i3} X_{i2}}}{1+ e^{\beta_0 + \beta_3 X_{i3} + \beta_2 X_{i2} + \beta_5 X_{i3} X_{i2}}} \\  
\text{priors:} & & \beta_0  &  \sim N\left(0, 0.5^2 \right)  \\  
							 & & \beta_2  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_3  & \sim N\left(0, 0.5^2 \right)\\  
						  & & \beta_5  & \sim N\left(0, 0.5^2 \right)\\  
\end{array}  
$$  
* **model3:  (激励和来源的作用，无交互)**  
$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1,\beta_2,\beta_3,\beta_4,\beta_5 & \stackrel{ind}{\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 + \beta_1 X_{i1} + \beta_2 X_{i2} + \beta_3 X_{i3} + \beta_4 X_{i1} X_{i2} + \beta_5 X_{i3} X_{i2}}}{1 + e^{\beta_0 + \beta_1 X_{i1} + \beta_2 X_{i2} + \beta_3 X_{i3} + \beta_4 X_{i1} X_{i2} + \beta_5 X_{i3} X_{i2}}} \\  
\text{priors:} & & \beta_0  &  \sim N\left(0, 0.5^2 \right)  \\  
               & & \beta_1  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_2  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_3  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_4  & \sim N\left(0, 0.5^2 \right)\\  
							& & \beta_5  & \sim N\left(0, 0.5^2 \right)\\  
\end{array}  
$$  
* **model4:  (激励和来源的作用，存在交互，指标为三者的交互效应)**  
$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1,\beta_2,\beta_3,\beta_4,\beta_5 \beta_6 & \stackrel{ind}{\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 + \beta_1 X_{i1} + \beta_2 X_{i2} + \beta_3 X_{i3}+ \beta_4 X_{i1} X_{i2} + \beta_5 X_{i3} X_{i2} + \beta_6 X_{i1} X_{i3} X_{i2}}}{1 + e^{\beta_0 + \beta_1 X_{i1} + \beta_2 X_{i2}+ \beta_3 X_{i3}+ \beta_4 X_{i1} X_{i2}+ \beta_5 X_{i3} X_{i2} +  + \beta_6 X_{i1} X_{i3} X_{i2}}} \\  
\text{priors:} & & \beta_0  &  \sim N\left(0, 0.5^2 \right)  \\  
                & & \beta_1  & \sim N\left(0, 0.5^2 \right)\\  
					 		  & & \beta_2  & \sim N\left(0, 0.5^2 \right)\\  
							  & & \beta_3  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_4  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_5  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_6  & \sim N\left(0, 0.5^2 \right)\\  
\end{array}  
$$  

* **由于因变量（被试判断结果）是一个二分变量（0，1），我们选取了Bernoulli分布来构建模型**

In [8]:
with pm.Model(coords={"obs_id": df.index}) as model1:
    
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)          #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)          #定义beta_2
    beta_4 = pm.Normal("beta_4", mu=0, sigma=0.5)          #定义beta_4

    acc = pm.MutableData("Accuracy",df.Accuracy, dims="obs_id")                         
    timu = pm.MutableData("timu",df.timu, dims="obs_id")
    
    # 预测 lambda，自变量与先验结合
    mu = pm.Deterministic("mu", beta_0 + beta_1*acc + beta_2*timu + beta_4*acc*timu,
                          dims="obs_id") 
                          
    #解释转换函数
    pi = pm.Deterministic("pi",pm.math.invlogit(mu),dims="obs_id") 
    likelihood = pm.Bernoulli("y_est", p = pi ,observed=df.value, dims="obs_id")

*可视化的图片*

In [9]:
pm.model_to_graphviz(model1)

In [10]:
with pm.Model(coords={"obs_id": df.index}) as model2:
    
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_2 = pm.Normal("beta_3", mu=0, sigma=0.5)          #定义beta_2
    beta_3 = pm.Normal("beta_2", mu=0, sigma=0.5)          #定义beta_3
    beta_5 = pm.Normal("beta_5", mu=0, sigma=0.5)          #定义beta_5

    sor = pm.MutableData("Source",df.Source, dims="obs_id")                        
    timu = pm.MutableData("timu",df.timu, dims="obs_id")
    
    # 预测 lambda，自变量与先验结合
    mu = pm.Deterministic("mu", beta_0 + beta_3*sor + beta_2*timu + beta_5 *sor*timu,
                          dims="obs_id")
    
    #解释转换函数
    pi = pm.Deterministic("pi",pm.math.invlogit(mu),dims="obs_id") 
    likelihood = pm.Bernoulli("y_est", p = pi ,observed=df.value, dims="obs_id")

In [11]:
pm.model_to_graphviz(model2)

In [12]:
with pm.Model(coords={"obs_id": df.index}) as model3:

    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)          #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)          #定义beta_2
    beta_3 = pm.Normal("beta_3", mu=0, sigma=0.5)          #定义beta_3          
    beta_4 = pm.Normal("beta_4", mu=0, sigma=0.5)          #定义beta_4
    beta_5 = pm.Normal("beta_5", mu=0, sigma=0.5)          #定义beta_5
    
    
    acc = pm.MutableData("Accuracy",df.Accuracy, dims="obs_id")                     
    sor = pm.MutableData("Source",df.Source, dims="obs_id")      
    timu = pm.MutableData("timu",df.timu, dims="obs_id")
    
    
    
    # 预测 lambda，自变量与先验结合
    mu = pm.Deterministic("mu", beta_0 + beta_1*acc + beta_3*sor + beta_2*timu + beta_4*acc*timu + beta_5*sor*timu,
                          dims="obs_id")
   
   #解释转换函数
    pi = pm.Deterministic("pi",pm.math.invlogit(mu),dims="obs_id") 
    likelihood = pm.Bernoulli("y_est", p = pi ,observed=df.value, dims="obs_id")

In [13]:
pm.model_to_graphviz(model3)

In [14]:
with pm.Model(coords={"obs_id": df.index}) as model4:

    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)          #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)          #定义beta_2
    beta_3 = pm.Normal("beta_3", mu=0, sigma=0.5)          #定义beta_0          
    beta_4 = pm.Normal("beta_4", mu=0, sigma=0.5)          #定义beta_4
    beta_5 = pm.Normal("beta_5", mu=0, sigma=0.5)          #定义beta_5
    beta_6 = pm.Normal("beta_6", mu=0, sigma=0.5)          #定义beta_6
    
    
    acc = pm.MutableData("Accuracy",df.Accuracy, dims="obs_id")                     
    sor = pm.MutableData("Source",df.Source, dims="obs_id")      
    timu = pm.MutableData("timu",df.timu, dims="obs_id")
    
    
    
    # 预测 lambda，自变量与先验结合
    mu = pm.Deterministic("mu", beta_0 + beta_1*acc + beta_2*timu + beta_3*sor + beta_4*acc*timu + beta_5*sor*timu + beta_6*timu*sor*acc,
                          dims="obs_id")
    
    #解释转换函数
    pi = pm.Deterministic("pi",pm.math.invlogit(mu),dims="obs_id") 
    likelihood = pm.Bernoulli("y_est", p = pi ,observed=df.value, dims="obs_id")

In [15]:
pm.model_to_graphviz(model4)

## 2.2 MCMC采样 

In [16]:
with model1:
    trace1 = pm.sample(draws= 2500,                   # 使用mcmc方法进行采样，draws为采样次数
        tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
        chains=3,                     # 链数
        discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
        random_seed=84735) 

    # 后验预测
    ppc1 = pm.sample_posterior_predictive(trace1, random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (3 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2, beta_4]


Sampling 3 chains for 1_000 tune and 2_500 draw iterations (3_000 + 7_500 draws total) took 9 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
Sampling: [y_est]


In [17]:
az.plot_trace(trace1, var_names=["beta_0","beta_1","beta_2","beta_4"],
              figsize=(15,15),
              compact=False)
plt.show()

<Figure size 1500x1500 with 8 Axes>

In [35]:
az.summary(trace1,
           var_names=["beta","beta_1","beta_2","beta_4"],
           filter_vars="like")

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,-0.464,0.038,-0.534,-0.394,0.000,0.000,13918.0,6508.0,1.0
beta_1,0.142,0.073,0.003,0.277,0.001,0.001,11072.0,6622.0,1.0
beta_2,1.274,0.075,1.135,1.416,0.001,0.000,11597.0,5804.0,1.0
beta_4,0.308,0.146,0.040,0.591,0.001,0.001,11212.0,6088.0,1.0


In [19]:
with model2:
    trace2 = pm.sample(
        draws=2500,                   # 使用mcmc方法进行采样，draws为采样次数
        tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
        chains=3,                     # 链数
        discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
        random_seed=84735) 

    # 后验预测
    ppc2 = pm.sample_posterior_predictive(trace2, random_seed=84735)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (3 chains in 4 jobs)
NUTS: [beta_0, beta_3, beta_2, beta_5]


Sampling 3 chains for 1_000 tune and 2_500 draw iterations (3_000 + 7_500 draws total) took 8 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
Sampling: [y_est]


In [20]:
az.plot_trace(trace2, var_names=["beta_0","beta_2","beta_3","beta_5"],
              figsize=(15,15),
              compact=False)
plt.show()

<Figure size 1500x1500 with 8 Axes>

In [36]:
az.summary(trace2,
           var_names=["beta","beta_2","beta_3","beta_5"],
           filter_vars="like")

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,-0.470,0.038,-0.541,-0.399,0.000,0.000,13133.0,6020.0,1.0
beta_3,1.271,0.075,1.136,1.414,0.001,0.000,11521.0,5830.0,1.0
beta_2,-0.119,0.076,-0.258,0.026,0.001,0.001,9615.0,5981.0,1.0
beta_5,0.243,0.144,-0.034,0.508,0.001,0.001,12018.0,6446.0,1.0


*在这里可以加一些mcmc采样后三条链拟合程度的图片*

In [22]:
with model3:
    trace3 = pm.sample(
        draws=2500,                   # 使用mcmc方法进行采样，draws为采样次数
        tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
        chains=3,                     # 链数
        discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
        random_seed=84735) 

    # 后验预测
    ppc3 = pm.sample_posterior_predictive(trace3, random_seed=84735)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (3 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2, beta_3, beta_4, beta_5]


Sampling 3 chains for 1_000 tune and 2_500 draw iterations (3_000 + 7_500 draws total) took 13 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
Sampling: [y_est]


In [23]:
az.plot_trace(trace3, var_names=["beta_0","beta_1","beta_2","beta_3","beta_4","beta_5"],
              figsize=(15,15),
              compact=False)
plt.show()

<Figure size 1500x1500 with 12 Axes>

In [24]:
az.summary(trace3,
           var_names=["beta","beta_1","beta_2","beta_3","beta_4","beta_5"],
           filter_vars="like")

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,-0.466,0.038,-0.541,-0.399,0.000,0.000,8616.0,6044.0,1.0
beta_1,0.146,0.076,0.006,0.290,0.001,0.001,9078.0,5940.0,1.0
beta_2,1.280,0.076,1.139,1.424,0.001,0.001,11237.0,6830.0,1.0
beta_3,-0.123,0.076,-0.267,0.020,0.001,0.001,9881.0,6265.0,1.0
beta_4,0.300,0.148,0.019,0.578,0.001,0.001,10023.0,5814.0,1.0
beta_5,0.234,0.147,-0.028,0.523,0.002,0.001,8769.0,6151.0,1.0


In [25]:
with model4:
    trace4 = pm.sample(
        draws=2500,                   # 使用mcmc方法进行采样，draws为采样次数
        tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
        chains=3,                     # 链数
        discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
        random_seed=84735) 

    # 后验预测
    ppc4 = pm.sample_posterior_predictive(trace4, random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (3 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2, beta_3, beta_4, beta_5, beta_6]


Sampling 3 chains for 1_000 tune and 2_500 draw iterations (3_000 + 7_500 draws total) took 15 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
Sampling: [y_est]


In [26]:
az.plot_trace(trace4, var_names=["beta_0","beta_1","beta_2","beta_3","beta_4","beta_5","beta_6"],
              figsize=(15,15),
              compact=False)
plt.show()

<Figure size 1500x1500 with 14 Axes>

# 3 后验解释

In [27]:
az.summary(trace4,
           var_names=["beta","beta_1","beta_2","beta_3","beta_4","beta_5","beta_6"],
           filter_vars="like")

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,-0.467,0.038,-0.536,-0.392,0.000,0.000,10158.0,6242.0,1.0
beta_1,0.144,0.076,0.000,0.288,0.001,0.001,8434.0,5649.0,1.0
beta_2,1.280,0.076,1.135,1.421,0.001,0.001,9679.0,5780.0,1.0
beta_3,-0.123,0.076,-0.270,0.018,0.001,0.001,9065.0,5923.0,1.0
beta_4,0.304,0.146,0.022,0.568,0.001,0.001,10013.0,6168.0,1.0
beta_5,0.235,0.147,-0.057,0.497,0.001,0.001,11312.0,6437.0,1.0
beta_6,0.082,0.259,-0.402,0.582,0.003,0.003,9832.0,6640.0,1.0


In [28]:
az.plot_posterior(trace4, var_names = ["beta_0","beta_1","beta_2","beta_3","beta_4","beta_5","beta_6"])

array([[<Axes: title={'center': 'beta_0'}>,
        <Axes: title={'center': 'beta_1'}>,
        <Axes: title={'center': 'beta_2'}>],
       [<Axes: title={'center': 'beta_3'}>,
        <Axes: title={'center': 'beta_4'}>,
        <Axes: title={'center': 'beta_5'}>],
       [<Axes: title={'center': 'beta_6'}>, <Axes: >, <Axes: >]],
      dtype=object)

<Figure size 2208x1656 with 9 Axes>

### 后验参数解释  

以下的结果显示：  
- $\beta_0 = -0.47$，那么 $e^{\beta_0} = 0.63$， 表明所有 X 值不存在时，个体将新闻判断为真与判断为假的概率比的可能性为 0.63。  

- $\beta_1 = 0.14$， $e^{\beta_1} = 1.15$  
   相比控制情况下，有准确性激励的情况下，该概率比上升为原来的1.15倍。  
	 
- $\beta_2 = 1.30$， $e^{\beta_2} = 3.67$  
  相比原题目是假的情况下，题目是真的情况下，该概率比上升为原来的3.67倍。  
	
- $\beta_4 = 0.30$， 表明被试对新闻的真假进行判断时，不仅受到新闻本身真假的影响，还受到是否存在激励条件的影响：  
  $e^{{\beta_4}{x1 = 0.5}} = 0.86$，在控制条件下，当新闻为真时，该概率比缩小为原来的0.86倍。  
	$e^{{\beta_4}{x1 = 0.5}} = 1.16$，在激励条件下，当新闻为真时，该概率比扩大为原来的1.16倍。  
	验证了本研究的假设1。  
	
- 然而，其他的beta值在94%的HDI中皆包含0，说明不存在其他显著的主效应和交互效应。

# 4 模型评估与比较

## 4.1 后验预测分布  

从后验预测的结果看，四个模型都能较为准确地描绘出观测数据的形态

In [29]:
az.plot_ppc(ppc1, num_pp_samples=500)
az.plot_ppc(ppc2, num_pp_samples=500)
az.plot_ppc(ppc3, num_pp_samples=500)
az.plot_ppc(ppc4, num_pp_samples=500)

<Axes: xlabel='y_est / y_est'>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

## 4.2 MAE  


MAE的结果显示：  
- 模型3的预测误差最小，即模型3的预测能力最优，其次分别为模型1和模型4，模型2的预测能力最差  
- 这与对后验参数的解释保持了一致：信息源的主效应不显著，准确性激励和信息源的交互作用不显著

In [30]:
# 定义计算 MAE 函数
from statistics import median
def MAE(model_ppc):
    # 计算每个X取值下对应的后验预测模型的均值
    pre_x = model_ppc.posterior_predictive["y_est"].stack(sample=("chain", "draw"))
    pre_y_mean = pre_x.mean(axis=1).values

    # 提取观测值Y，提取对应Y值下的后验预测模型的均值
    MAE = pd.DataFrame({
        "value_ppc_mean": pre_y_mean,
        "value_original": df.value
    })

    # 计算预测误差
    MAE["pre_error"] = abs(MAE["value_original"] -\
                            MAE["value_ppc_mean"])

    # 最后，计算预测误差的中位数
    MAE = median(MAE.pre_error)
    return MAE

# 输出结果
model1_MAE = MAE(ppc1)
print(f"模型1 MAE: {model1_MAE:.3f}")
model2_MAE = MAE(ppc2)
print(f"模型2 MAE: {model2_MAE:.3f}")
model3_MAE = MAE(ppc3)
print(f"模型3 MAE: {model3_MAE:.3f}")
model4_MAE = MAE(ppc4)
print(f"模型4 MAE: {model4_MAE:.3f}")

模型1 MAE: 0.428
模型2 MAE: 0.458
模型3 MAE: 0.427
模型4 MAE: 0.428


## 4.3 ELPD  

通过 `arviz.compare` 方法来对比多个模型的 elpd，面结果可见：  
- 模型3的 elpd_loo 最大，表明它对**样本外数据**的预测性能最好。  
- 而模型2的 elpd_loo 最小，表明它的预测性能最差。  
- 这些结果与我们通过 MAE 和 后验预测区间得到的判断一致。  

需要注意的是：  
- arviz 提供的结果包括了 elpd se，这使得我们可以判断两个模型的预测差异 elpd_diff 是否超过两至三个标准误se。  
- 从现在的结果看，四个模型间的差异均小于两个标准误se很小，表明四个模型的预测性能差异并不显著。

In [31]:
with model1:
    pm.compute_log_likelihood(trace1)

with model2:
    pm.compute_log_likelihood(trace2)  

with model3:
    pm.compute_log_likelihood(trace3) 

with model4:
    pm.compute_log_likelihood(trace4)

comparison_list = {"model1":trace1,"model2":trace2,"model3":trace3,"model4":trace4,}
az.compare(comparison_list)

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
model3,0,-1994.866226,5.744454,0.000000,0.425524,19.906767,0.000000,False,log
model1,1,-1995.296814,3.728636,0.430588,0.397671,19.828318,2.065504,False,log
model4,2,-1995.506089,6.450553,0.639863,0.000000,19.935874,0.271462,False,log
model2,3,-1997.657970,3.772173,2.791744,0.176804,19.692912,2.942382,False,log


# 5 后验模型可视化

综合对后验参数的理解和比较，我们选择模型1作为最后适用的模型，并对其进行可视化

In [32]:
T_index = xr.DataArray(df.query("timu == 0.5")["Accuracy"])["index"].values
F_index = xr.DataArray(df.query("timu == -0.5")["Accuracy"])["index"].values

In [33]:
T_para = trace1.sel(obs_id=T_index)
F_para = trace1.sel(obs_id=F_index)

In [34]:
def plot_ppc_lm(T_para, F_para):
    fig, axes = plt.subplots(figsize=(10, 8))
    
az.plot_hdi(
    T_para.constant_data.Accuracy,
    T_para.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="C1"
)
#得到每个自变量对应的因变量概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean1 = T_para.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = T_para.constant_data.Accuracy,
             y= post_mean1, 
             label="T:posterior mean", 
             color="C1")
#绘制真实数据散点图
sns.scatterplot(x = T_para.constant_data.Accuracy,
                y=  T_para.observed_data.y_est,
                label="T:observed data", 
                color='C1', 
                alpha=0.5)
                
az.plot_hdi(
    F_para.constant_data.Accuracy,
    F_para.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="C2"
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean2 = F_para.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = F_para.constant_data.Accuracy,
             y= post_mean2, 
             label="F:posterior mean", 
             color="C2")
#绘制真实数据散点图
sns.scatterplot(x = F_para.constant_data.Accuracy,
                y=  F_para.observed_data.y_est,
                label="F:observed data", 
                color='C2', 
                alpha=0.5)

<Axes: xlabel='Accuracy', ylabel='pi'>

<Figure size 640x480 with 1 Axes>

# 6 结论  
* 1 准确性激励显著影响被试判断的准确性，具体表现为存在准确性激励时判断更准确、不存在准确性激励时判断更不准确  
* 2 信息源对被试判断准确性的影响不显著，交互作用不显著  
* 3 相比原方法，发现准确性激励对被试判断偏向的主效应，即当存在准确性激励时，被试更倾向于将新闻判断为真新闻，这可能暗示了准确性动机的内在作用机制  
* 4 相比原方法，贝叶斯方法能在样本量有限的情况下对变量间的交互作用进行检验  
	